In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
# visualization libs
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# import train data

train_df = pd.read_csv('/kaggle/input/titanic/train.csv')
test_df = pd.read_csv('/kaggle/input/titanic/test.csv')
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# check for null value

train_df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
test_df.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [6]:
# Fill in missing values for columns 'Age' & 'Embarked'

train_df['Age'].fillna(train_df['Age'].median(), inplace=True)
test_df['Age'].fillna(train_df['Age'].median(), inplace=True)
test_df['Fare'].fillna(test_df['Fare'].median(), inplace=True)

In [7]:
train_df.Embarked.mode()

0    S
dtype: object

In [8]:
# Since the most common value in 'Embarked' column is 'S', change all missing values to this

most_value = train_df.Embarked.mode().iloc[0]
train_df['Embarked'].fillna(most_value, inplace=True)

# check for null value again
train_df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [9]:
test_df.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64

In [10]:
# Let's remove insignificant features 'Name', 'Cabin' & 'Ticket' and extract the 'Survived' column

train_y = train_df['Survived']

train_df.drop(['Name', 'Ticket', 'Cabin', 'Survived'], axis=1, inplace=True)
test_df.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
train_df

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,3,male,22.0,1,0,7.2500,S
1,2,1,female,38.0,1,0,71.2833,C
2,3,3,female,26.0,0,0,7.9250,S
3,4,1,female,35.0,1,0,53.1000,S
4,5,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,887,2,male,27.0,0,0,13.0000,S
887,888,1,female,19.0,0,0,30.0000,S
888,889,3,female,28.0,1,2,23.4500,S
889,890,1,male,26.0,0,0,30.0000,C


In [11]:
test_df

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,male,34.5,0,0,7.8292,Q
1,893,3,female,47.0,1,0,7.0000,S
2,894,2,male,62.0,0,0,9.6875,Q
3,895,3,male,27.0,0,0,8.6625,S
4,896,3,female,22.0,1,1,12.2875,S
...,...,...,...,...,...,...,...,...
413,1305,3,male,28.0,0,0,8.0500,S
414,1306,1,female,39.0,0,0,108.9000,C
415,1307,3,male,38.5,0,0,7.2500,S
416,1308,3,male,28.0,0,0,8.0500,S


In [12]:
# As you can see, there are categorical signs in the dataframe that need to be converted to numerical ones. 
# Since there are not many different options for values in these columns, 
# I will use the standard OrdinalEncoder.

from sklearn.preprocessing import OrdinalEncoder as OE

cat = ['Sex', 'Embarked']

encoder = OE()
encoder.fit(train_df[cat])
train_df[cat] = encoder.transform(train_df[cat])
test_df[cat] = encoder.transform(test_df[cat])
train_df

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,3,1.0,22.0,1,0,7.2500,2.0
1,2,1,0.0,38.0,1,0,71.2833,0.0
2,3,3,0.0,26.0,0,0,7.9250,2.0
3,4,1,0.0,35.0,1,0,53.1000,2.0
4,5,3,1.0,35.0,0,0,8.0500,2.0
...,...,...,...,...,...,...,...,...
886,887,2,1.0,27.0,0,0,13.0000,2.0
887,888,1,0.0,19.0,0,0,30.0000,2.0
888,889,3,0.0,28.0,1,2,23.4500,2.0
889,890,1,1.0,26.0,0,0,30.0000,0.0


In [13]:
test_df

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,1.0,34.5,0,0,7.8292,1.0
1,893,3,0.0,47.0,1,0,7.0000,2.0
2,894,2,1.0,62.0,0,0,9.6875,1.0
3,895,3,1.0,27.0,0,0,8.6625,2.0
4,896,3,0.0,22.0,1,1,12.2875,2.0
...,...,...,...,...,...,...,...,...
413,1305,3,1.0,28.0,0,0,8.0500,2.0
414,1306,1,0.0,39.0,0,0,108.9000,0.0
415,1307,3,1.0,38.5,0,0,7.2500,2.0
416,1308,3,1.0,28.0,0,0,8.0500,2.0


In [14]:
df = train_df.drop(['PassengerId'], axis=1)
df

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1.0,22.0,1,0,7.2500,2.0
1,1,0.0,38.0,1,0,71.2833,0.0
2,3,0.0,26.0,0,0,7.9250,2.0
3,1,0.0,35.0,1,0,53.1000,2.0
4,3,1.0,35.0,0,0,8.0500,2.0
...,...,...,...,...,...,...,...
886,2,1.0,27.0,0,0,13.0000,2.0
887,1,0.0,19.0,0,0,30.0000,2.0
888,3,0.0,28.0,1,2,23.4500,2.0
889,1,1.0,26.0,0,0,30.0000,0.0


In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, train_y, train_size=0.7, 
                                                    test_size=0.3, random_state=0)

In [16]:
# Let's train several models to understand which model has the best quality

from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

clf0 = RandomForestClassifier(n_estimators=1000, 
                              random_state=0)
clf0.fit(X_train, y_train)
predict0 = clf0.predict(X_test)

print(accuracy_score(y_test, predict0))

0.8283582089552238


In [17]:
from sklearn.tree import DecisionTreeClassifier
clf1 = DecisionTreeClassifier(random_state=0,
                              max_leaf_nodes=100,
                              max_depth=100)
clf1.fit(X_train, y_train)
predict1 = clf1.predict(X_test)

print(accuracy_score(y_test, predict1))

0.7798507462686567


In [18]:
import xgboost as xgb
clf2 = xgb.XGBClassifier(learning_rate=0.1,
                         n_estimators= 200,
                         objective= 'binary:logistic',
                         max_depth= 2000)

clf2.fit(X_train, y_train)
predict2 = clf2.predict(X_test)

print(accuracy_score(y_test, predict2))

[16:12:17] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.8171641791044776


In [19]:
# It can be seen that Random Forest has the highest accuracy score.
# Let's try to improve the quality using a grid of parameters.

clssifier = RandomForestClassifier(random_state=0)
clssifier.get_params().keys()

dict_keys(['bootstrap', 'ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

In [20]:
from sklearn.model_selection import GridSearchCV

param_grid = { 
    'n_estimators': [200, 300, 400, 500, 1000],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [6, 7, 8, 10],
    'criterion' : ['gini', 'entropy']
}

clf_cv = GridSearchCV(estimator=clssifier, param_grid=param_grid, cv=5)
clf_cv.fit(df, train_y) 

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=0),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [6, 7, 8, 10],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [200, 300, 400, 500, 1000]})

In [21]:
clf_cv.best_params_

{'criterion': 'entropy',
 'max_depth': 10,
 'max_features': 'auto',
 'n_estimators': 300}

In [22]:
clf_cv.best_score_

0.8350260498399347

In [23]:
rfc_check = RandomForestClassifier(criterion='gini',
                                  max_depth=8,
                                  max_features='auto',
                                  n_estimators=200,
                                  random_state=2019)
test_df

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,1.0,34.5,0,0,7.8292,1.0
1,893,3,0.0,47.0,1,0,7.0000,2.0
2,894,2,1.0,62.0,0,0,9.6875,1.0
3,895,3,1.0,27.0,0,0,8.6625,2.0
4,896,3,0.0,22.0,1,1,12.2875,2.0
...,...,...,...,...,...,...,...,...
413,1305,3,1.0,28.0,0,0,8.0500,2.0
414,1306,1,0.0,39.0,0,0,108.9000,0.0
415,1307,3,1.0,38.5,0,0,7.2500,2.0
416,1308,3,1.0,28.0,0,0,8.0500,2.0


In [24]:
rfc_check.fit(X_test, y_test)

RandomForestClassifier(max_depth=8, n_estimators=200, random_state=2019)

In [25]:
pred_check = rfc_check.predict(X_test)
print(accuracy_score(y_test, pred_check))

0.9626865671641791


In [26]:
res = pd.DataFrame(test_df['PassengerId'])
test_df.drop(['PassengerId'], axis=1, inplace=True)

rfc_main = RandomForestClassifier(criterion='gini',
                                  max_depth=8,
                                  max_features='auto',
                                  n_estimators=200,
                                  random_state=2019)
test_df

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1.0,34.5,0,0,7.8292,1.0
1,3,0.0,47.0,1,0,7.0000,2.0
2,2,1.0,62.0,0,0,9.6875,1.0
3,3,1.0,27.0,0,0,8.6625,2.0
4,3,0.0,22.0,1,1,12.2875,2.0
...,...,...,...,...,...,...,...
413,3,1.0,28.0,0,0,8.0500,2.0
414,1,0.0,39.0,0,0,108.9000,0.0
415,3,1.0,38.5,0,0,7.2500,2.0
416,3,1.0,28.0,0,0,8.0500,2.0


In [27]:
rfc_main.fit(df, train_y)

RandomForestClassifier(max_depth=8, n_estimators=200, random_state=2019)

In [28]:
pred_main = rfc_main.predict(test_df)

In [29]:
res['Survived'] = pred_main

res.to_csv('res_rfc.csv', index=False)